# File and libraries

In [92]:
import pandas as pd
import numpy as np


file_tag = "Google Merch Store 2024-2025"

# DSLabs functions

In [93]:
%run "scripts/dslabs_functions.py"


# data functions

In [94]:
%run "scripts/data_functions.py"


data_functions loaded


# Load and sampling

In [95]:
# test_data=True
test_data=False


# Define a function to sample 10% from each group
def sample_per_day(group, fraction=0.1):
    return group.sample(frac=fraction)


# gsc_data = pd.read_csv(r'data\search_console_google_merch_2024_2025.csv')    # too much missing data

total_data = pd.read_csv(r'data\google_merch_total_sessions_2024_2025.csv')
additional_data = pd.read_csv(r'data\google_merch_total_sessions_2024_2025_additional_metrics.csv')
mobile_data=pd.read_csv(r'data\google_merch_mobile_sessions_2024_2025.csv')

data=total_data.merge(additional_data, on='Date', how='left')

data=data.merge(mobile_data, on='Date', how='left', suffixes=('', ' Mobile'))


# parse Date as datetime
data['Date'] = pd.to_datetime(data['Date'])

# Display the information of the merged dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427 entries, 0 to 426
Data columns (total 51 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   Date                                      427 non-null    datetime64[ns]
 1   Sessions                                  427 non-null    int64         
 2   Bounce rate                               427 non-null    float64       
 3   Seven-day active users                    427 non-null    int64         
 4   28-day active users                       427 non-null    int64         
 5   New users                                 427 non-null    int64         
 6   Scrolled users                            427 non-null    int64         
 7   Engaged sessions                          427 non-null    int64         
 8   Views                                     427 non-null    int64         
 9   Event count                     

## session medium time series and join to data

In [96]:
session_medium_data=pd.read_csv(r'data\google_merch_session_medium_time_series.csv')

session_medium_data['Date'] = pd.to_datetime(session_medium_data['Date'])


# pivot the data to have session medium as columns over Sessions column and purchases column
session_medium_data = session_medium_data.pivot(index='Date', columns='Session medium', values=['Sessions'])

# rename columns to sessions_session_medium and purchases_session_medium from variables name
session_medium_data.columns = [' '.join(col).strip() for col in session_medium_data.columns.values]

# missing values to 0
session_medium_data = session_medium_data.fillna(0)

session_medium_data

,Sessions (none),Sessions (not set),Sessions affiliate,Sessions cpc,Sessions cpm,Sessions email,Sessions organic,Sessions other,Sessions referral
Date,,,,,,,,,
2024-01-04,535.0,53.0,0.0,34.0,0.0,45.0,263.0,0.0,50.0
2024-01-05,512.0,45.0,0.0,17.0,0.0,32.0,234.0,0.0,45.0
2024-01-06,240.0,19.0,0.0,23.0,0.0,6.0,131.0,0.0,6.0
2024-01-07,243.0,26.0,0.0,23.0,0.0,7.0,135.0,0.0,11.0
2024-01-08,545.0,47.0,0.0,22.0,0.0,43.0,269.0,0.0,56.0
...,...,...,...,...,...,...,...,...,...
2025-03-01,1046.0,18.0,0.0,76.0,0.0,2.0,118.0,0.0,39.0
2025-03-02,685.0,11.0,0.0,104.0,0.0,8.0,160.0,0.0,32.0
2025-03-03,747.0,25.0,0.0,150.0,0.0,39.0,315.0,0.0,181.0


In [97]:
# join the data with session_medium_data
data = data.merge(session_medium_data, on='Date', how='left')
data

,Date,Sessions,Bounce rate,Seven-day active users,28-day active users,New users,Scrolled users,Engaged sessions,Views,Event count,...,Average session duration Mobile,Sessions (none),Sessions (not set),Sessions affiliate,Sessions cpc,Sessions cpm,Sessions email,Sessions organic,Sessions other,Sessions referral
0,2025-03-05,1304,0.368098,6906,29095,627,620,824,6155,23166,...,321.875012,717.0,34.0,0.0,162.0,0.0,21.0,291.0,0.0,78.0
1,2025-03-04,1538,0.379714,7047,29494,791,723,954,6664,25808,...,215.443322,830.0,28.0,0.0,207.0,0.0,26.0,300.0,0.0,147.0
2,2025-03-03,1451,0.352860,7158,29497,674,710,939,6884,26862,...,159.868685,747.0,25.0,0.0,150.0,0.0,39.0,315.0,0.0,181.0
3,2025-03-02,1002,0.469062,7404,29753,692,337,532,3582,14427,...,169.675856,685.0,11.0,0.0,104.0,0.0,8.0,160.0,0.0,32.0
4,2025-03-01,1298,0.554700,7458,29843,1091,325,578,3392,14491,...,120.551822,1046.0,18.0,0.0,76.0,0.0,2.0,118.0,0.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,2024-01-08,981,0.387360,2749,2749,428,0,601,4745,16848,...,190.422588,545.0,47.0,0.0,22.0,0.0,43.0,269.0,0.0,56.0
423,2024-01-07,446,0.482063,2081,2081,261,0,231,2129,8848,...,152.000064,243.0,26.0,0.0,23.0,0.0,7.0,135.0,0.0,11.0
424,2024-01-06,426,0.415493,1748,1748,256,0,249,1920,6913,...,193.068948,240.0,19.0,0.0,23.0,0.0,6.0,131.0,0.0,6.0
425,2024-01-05,887,0.363021,1433,1433,427,0,565,5000,17829,...,296.515292,512.0,45.0,0.0,17.0,0.0,32.0,234.0,0.0,45.0


## shape

In [98]:
data.shape

(427, 60)

In [99]:
summary5 = data.describe(include="all")

summary5

,Date,Sessions,Bounce rate,Seven-day active users,28-day active users,New users,Scrolled users,Engaged sessions,Views,Event count,...,Average session duration Mobile,Sessions (none),Sessions (not set),Sessions affiliate,Sessions cpc,Sessions cpm,Sessions email,Sessions organic,Sessions other,Sessions referral
count,427,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,...,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000,427.000000
mean,2024-08-04 00:00:00.000000256,2096.981265,0.477794,10567.112412,38404.140515,1208.053864,642.749415,1129.370023,7518.901639,29223.332553,...,142.810721,1431.451991,49.114754,0.002342,95.847775,0.470726,65.601874,286.948478,1.653396,164.238876
min,2024-01-04 00:00:00,22.000000,0.165997,780.000000,780.000000,4.000000,0.000000,4.000000,20.000000,401.000000,...,0.957704,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000
25%,2024-04-19 12:00:00,1463.000000,0.412334,8326.500000,30502.500000,876.500000,264.000000,662.500000,4267.000000,16799.000000,...,111.140177,981.500000,25.000000,0.000000,18.000000,0.000000,5.000000,180.500000,0.000000,36.000000
50%,2024-08-04 00:00:00,1984.000000,0.478213,10800.000000,40024.000000,1177.000000,599.000000,999.000000,6563.000000,25377.000000,...,132.452059,1467.000000,37.000000,0.000000,71.000000,0.000000,16.000000,280.000000,0.000000,75.000000
75%,2024-11-18 12:00:00,2545.500000,0.545998,12269.500000,48328.000000,1442.000000,874.500000,1346.500000,9370.500000,36457.500000,...,162.904101,1784.000000,60.000000,0.000000,165.500000,0.000000,44.000000,347.500000,3.000000,181.000000
max,2025-03-05 00:00:00,11239.000000,0.818182,20447.000000,60050.000000,7734.000000,6455.000000,8453.000000,94665.000000,353391.000000,...,549.792113,9871.000000,235.000000,1.000000,360.000000,9.000000,2751.000000,1042.000000,14.000000,2663.000000
std,NaN,1078.436107,0.100940,3632.601576,12293.299375,627.503784,665.872685,776.451498,6176.753658,23458.915577,...,60.614302,747.508431,35.614043,0.048393,87.643365,1.153057,236.352051,138.374253,2.798685,264.319534


# replace false 0 as null (tracking issues)

In [100]:
from pandas import Series, to_numeric, to_datetime
def get_variable_types(df: DataFrame) -> dict[str, list]:
    variable_types: dict = {"numeric": [], "binary": [], "date": [], "symbolic": []}

    nr_values: Series = df.nunique(axis=0, dropna=True)
    for c in df.columns:
        if 2 == nr_values[c]:
            variable_types["binary"].append(c)
            df[c].astype("bool")
        else:
            try:
                to_numeric(df[c], errors="raise")
                variable_types["numeric"].append(c)
            except ValueError:
                try:
                    df[c] = to_datetime(df[c], errors="raise")
                    variable_types["date"].append(c)
                except ValueError:
                    variable_types["symbolic"].append(c)

    return variable_types

# Ensure 'Date' column is in datetime format before calling the function
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

variable_types: dict[str, list] = get_variable_types(data)
print(variable_types)

{'numeric': ['Date', 'Sessions', 'Bounce rate', 'Seven-day active users', '28-day active users', 'New users', 'Scrolled users', 'Engaged sessions', 'Views', 'Event count', 'Events per session', 'Item-list view events', 'Item view events', 'Item promotion click-through rate', 'Add to baskets', 'Checkouts', 'E-commerce purchases', 'Average purchase revenue', 'Purchase revenue', 'First time purchasers', 'Average session duration', 'ARPU', 'ARPPU', 'DAU/MAU', 'DAU/WAU', 'Sessions per user', 'Views per user', 'Total users', 'Promotion views', 'Promotion clicks', 'One-day active users', 'Sessions Mobile', 'Bounce rate Mobile', 'Seven-day active users Mobile', '28-day active users Mobile', 'New users Mobile', 'Scrolled users Mobile', 'Engaged sessions Mobile', 'Views Mobile', 'Event count Mobile', 'Events per session Mobile', 'Item-list view events Mobile', 'Item view events Mobile', 'Item promotion click-through rate Mobile', 'Add to baskets Mobile', 'Checkouts Mobile', 'E-commerce purchases

In [101]:
numeric: list[str] = [col for col in variable_types["numeric"] if col != "Date"]


# Exclude columns from session_medium_data
session_medium_cols = session_medium_data.columns.tolist()
numeric = [col for col in numeric if col not in session_medium_cols]


# Create a new dataframe excluding rows with Date beyond 2024-05-01
old_data = data[data['Date'] < '2024-05-01']
recent_data = data[data['Date'] >= '2024-05-01']


# Calculate mean and standard deviation for each column
mean_values = old_data[numeric].mean()
std_values = old_data[numeric].std()

# Identify columns with tracking issues based on sequence of values
def identify_tracking_issues(df, columns, threshold=0.08):
    tracking_issues = []
    for col in columns:
        # Calculate the proportion of zero values
        zero_proportion = (df[col] == 0).mean()
        # Check if the proportion of zero values exceeds the threshold
        if zero_proportion > threshold:
            tracking_issues.append(col)
    return tracking_issues

# Identify columns with tracking issues based on sequence of values
tracking_issue_columns_sequence = identify_tracking_issues(old_data, numeric)

# Replace 0 values in these columns with None
old_data[tracking_issue_columns_sequence] = old_data[tracking_issue_columns_sequence].replace(0, np.nan)

# Display the columns with tracking issues based on sequence of values
print("Columns with tracking issues based on sequence of values:", tracking_issue_columns_sequence)

# Concatenate old_data and recent_data
data = pd.concat([old_data, recent_data], ignore_index=True).sort_values(by='Date', ascending=True)

# Reset the index
data.reset_index(drop=True, inplace=True)

data

Columns with tracking issues based on sequence of values: ['Scrolled users', 'Item promotion click-through rate', 'E-commerce purchases', 'Purchase revenue', 'First time purchasers', 'ARPU', 'Promotion views', 'Promotion clicks', 'Scrolled users Mobile', 'Item promotion click-through rate Mobile', 'Checkouts Mobile', 'E-commerce purchases Mobile', 'Purchase revenue Mobile', 'First time purchasers Mobile']


,Date,Sessions,Bounce rate,Seven-day active users,28-day active users,New users,Scrolled users,Engaged sessions,Views,Event count,...,Average session duration Mobile,Sessions (none),Sessions (not set),Sessions affiliate,Sessions cpc,Sessions cpm,Sessions email,Sessions organic,Sessions other,Sessions referral
0,2024-01-04,982,0.362525,780,780,444,NaN,626,5111,19589,...,175.103036,535.0,53.0,0.0,34.0,0.0,45.0,263.0,0.0,50.0
1,2024-01-05,887,0.363021,1433,1433,427,NaN,565,5000,17829,...,296.515292,512.0,45.0,0.0,17.0,0.0,32.0,234.0,0.0,45.0
2,2024-01-06,426,0.415493,1748,1748,256,NaN,249,1920,6913,...,193.068948,240.0,19.0,0.0,23.0,0.0,6.0,131.0,0.0,6.0
3,2024-01-07,446,0.482063,2081,2081,261,NaN,231,2129,8848,...,152.000064,243.0,26.0,0.0,23.0,0.0,7.0,135.0,0.0,11.0
4,2024-01-08,981,0.387360,2749,2749,428,NaN,601,4745,16848,...,190.422588,545.0,47.0,0.0,22.0,0.0,43.0,269.0,0.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,2025-03-01,1298,0.554700,7458,29843,1091,325.0,578,3392,14491,...,120.551822,1046.0,18.0,0.0,76.0,0.0,2.0,118.0,0.0,39.0
423,2025-03-02,1002,0.469062,7404,29753,692,337.0,532,3582,14427,...,169.675856,685.0,11.0,0.0,104.0,0.0,8.0,160.0,0.0,32.0
424,2025-03-03,1451,0.352860,7158,29497,674,710.0,939,6884,26862,...,159.868685,747.0,25.0,0.0,150.0,0.0,39.0,315.0,0.0,181.0
425,2025-03-04,1538,0.379714,7047,29494,791,723.0,954,6664,25808,...,215.443322,830.0,28.0,0.0,207.0,0.0,26.0,300.0,0.0,147.0


## Time Features

In [102]:




# Categorize the season based on 'event_date' column
def categorize_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

data['season'] = data['Date'].apply(categorize_season)




season_encoding_mapping={
    'Winter':0,   
    'Spring':pi/2,
    'Summer':pi,
    'Fall':-pi/2,   
}


data['season'] = encode_column_with_mapping(data, 'season', season_encoding_mapping)


# Categorize the commercial/holiday season based on 'event_date' column
def categorize_commercial_season(date):
    month = date.month
    day = date.day
    if (month == 12 and day >= 20) or (month == 1 and day <= 5):
        return 'Christmas/New Year'
    elif (month == 11 and day >= 20) or (month == 12 and day < 20):
        return 'Black Friday/Cyber Monday'
    elif (month == 10 and day >= 20) or (month == 11 and day < 20):
        return 'Halloween'
    elif (month == 4 and day >= 10) or (month == 4 and day <= 20):
        return 'Easter'
    elif (month == 2 and day >= 10) or (month == 2 and day <= 20):
        return 'Valentine\'s Day'
    else:
        return 'Regular Season'

data['commercial_season'] = data['Date'].apply(categorize_commercial_season)


commercial_season_encoding_mapping = {
    'Christmas/New Year': 0,
    'Black Friday/Cyber Monday': pi/3,
    'Halloween': 2*pi/3,
    'Easter': pi,
    'Valentine\'s Day': -2*pi/3,
    'Regular Season': -pi/3
}

data['commercial_season'] = encode_column_with_mapping(data, 'commercial_season', commercial_season_encoding_mapping)

# create year, quarter, month, day number of week, weekend/weekday based on event_date column


# Assuming 'local_date' is in datetime format, otherwise you can parse it using pd.to_datetime
def week_of_month(dt):
    first_day = dt.replace(day=1)
    # Calculate the week of the month by comparing the current date to the first day of the month
    return (dt.day + first_day.weekday()) // 7 + 1

# Apply this function to your 'local_date' column
data['week_of_month'] = data['Date'].apply(week_of_month)


data['day_of_week_nr'] = data['Date'].dt.weekday  # Monday=0, Sunday=6
data['is_weekend'] = data['day_of_week_nr'].apply(lambda x: 1 if x >= 5 else 0)  # 1 for weekend, 0 for weekday


data

,Date,Sessions,Bounce rate,Seven-day active users,28-day active users,New users,Scrolled users,Engaged sessions,Views,Event count,...,Sessions cpm,Sessions email,Sessions organic,Sessions other,Sessions referral,season,commercial_season,week_of_month,day_of_week_nr,is_weekend
0,2024-01-04,982,0.362525,780,780,444,NaN,626,5111,19589,...,0.0,45.0,263.0,0.0,50.0,0.000000,0.000000,1,3,0
1,2024-01-05,887,0.363021,1433,1433,427,NaN,565,5000,17829,...,0.0,32.0,234.0,0.0,45.0,0.000000,0.000000,1,4,0
2,2024-01-06,426,0.415493,1748,1748,256,NaN,249,1920,6913,...,0.0,6.0,131.0,0.0,6.0,0.000000,-1.047198,1,5,1
3,2024-01-07,446,0.482063,2081,2081,261,NaN,231,2129,8848,...,0.0,7.0,135.0,0.0,11.0,0.000000,-1.047198,2,6,1
4,2024-01-08,981,0.387360,2749,2749,428,NaN,601,4745,16848,...,0.0,43.0,269.0,0.0,56.0,0.000000,-1.047198,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,2025-03-01,1298,0.554700,7458,29843,1091,325.0,578,3392,14491,...,0.0,2.0,118.0,0.0,39.0,1.570796,-1.047198,1,5,1
423,2025-03-02,1002,0.469062,7404,29753,692,337.0,532,3582,14427,...,0.0,8.0,160.0,0.0,32.0,1.570796,-1.047198,2,6,1
424,2025-03-03,1451,0.352860,7158,29497,674,710.0,939,6884,26862,...,0.0,39.0,315.0,0.0,181.0,1.570796,-1.047198,2,0,0
425,2025-03-04,1538,0.379714,7047,29494,791,723.0,954,6664,25808,...,0.0,26.0,300.0,0.0,147.0,1.570796,-1.047198,2,1,0


## us holidays

In [103]:
us_holidays = {
    2024: {
        'New Year\'s Day': '2024-01-01',
        'Martin Luther King Jr. Day': '2024-01-15',
        'Presidents\' Day': '2024-02-19',
        'Memorial Day': '2024-05-27',
        'Independence Day': '2024-07-04',
        'Labor Day': '2024-09-02',
        'Columbus Day': '2024-10-14',
        'Veterans Day': '2024-11-11',
        'Thanksgiving Day': '2024-11-28',
        'Christmas Day': '2024-12-25'
    },
    2025: {
        'New Year\'s Day': '2025-01-01',
        'Martin Luther King Jr. Day': '2025-01-20',
        'Presidents\' Day': '2025-02-17',
        'Memorial Day': '2025-05-26',
        'Independence Day': '2025-07-04',
        'Labor Day': '2025-09-01',
        'Columbus Day': '2025-10-13',
        'Veterans Day': '2025-11-11',
        'Thanksgiving Day': '2025-11-27',
        'Christmas Day': '2025-12-25'
    }
}

# Flatten the holidays dictionary into a single dictionary with dates as keys
holidays = {pd.to_datetime(date): name for year in us_holidays for name, date in us_holidays[year].items()}


# Create a new column 'is_holiday' that indicates whether each date is a holiday
data['is_holiday'] = data['Date'].apply(lambda x: 1 if x in holidays else 0)

# Display the DataFrame
data

,Date,Sessions,Bounce rate,Seven-day active users,28-day active users,New users,Scrolled users,Engaged sessions,Views,Event count,...,Sessions email,Sessions organic,Sessions other,Sessions referral,season,commercial_season,week_of_month,day_of_week_nr,is_weekend,is_holiday
0,2024-01-04,982,0.362525,780,780,444,NaN,626,5111,19589,...,45.0,263.0,0.0,50.0,0.000000,0.000000,1,3,0,0
1,2024-01-05,887,0.363021,1433,1433,427,NaN,565,5000,17829,...,32.0,234.0,0.0,45.0,0.000000,0.000000,1,4,0,0
2,2024-01-06,426,0.415493,1748,1748,256,NaN,249,1920,6913,...,6.0,131.0,0.0,6.0,0.000000,-1.047198,1,5,1,0
3,2024-01-07,446,0.482063,2081,2081,261,NaN,231,2129,8848,...,7.0,135.0,0.0,11.0,0.000000,-1.047198,2,6,1,0
4,2024-01-08,981,0.387360,2749,2749,428,NaN,601,4745,16848,...,43.0,269.0,0.0,56.0,0.000000,-1.047198,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,2025-03-01,1298,0.554700,7458,29843,1091,325.0,578,3392,14491,...,2.0,118.0,0.0,39.0,1.570796,-1.047198,1,5,1,0
423,2025-03-02,1002,0.469062,7404,29753,692,337.0,532,3582,14427,...,8.0,160.0,0.0,32.0,1.570796,-1.047198,2,6,1,0
424,2025-03-03,1451,0.352860,7158,29497,674,710.0,939,6884,26862,...,39.0,315.0,0.0,181.0,1.570796,-1.047198,2,0,0,0
425,2025-03-04,1538,0.379714,7047,29494,791,723.0,954,6664,25808,...,26.0,300.0,0.0,147.0,1.570796,-1.047198,2,1,0,0


## significant event days

In [104]:
significant_events = {
    '2024-02-11': 'Super Bowl LVIII',
    '2024-03-11': 'COVID-19 Pandemic Anniversary',
    '2024-07-15': 'Tax Day',
    '2024-11-05': 'US Presidential Election',
    '2024-11-28': 'Thanksgiving',
    '2024-12-25': 'Christmas',
    '2025-01-20': 'Inauguration Day',
    '2025-02-09': 'Super Bowl LIX',
    '2025-04-15': 'Tax Day',
    '2025-11-27': 'Thanksgiving',
    '2025-12-25': 'Christmas'
}

commercial_events = {
    '2024-11-29': 'Black Friday',
    '2024-12-02': 'Cyber Monday',
    '2025-06-23': 'Amazon Prime Day',
    '2025-06-24': 'Amazon Prime Day',
    '2025-11-28': 'Black Friday',
    '2025-12-01': 'Cyber Monday',
}



# Flatten the significant_events dictionary into a single dictionary with dates as keys
events = {pd.to_datetime(date): name for date, name in significant_events.items()}
commercial_events = {pd.to_datetime(date): name for date, name in commercial_events.items()}

# Create a new column 'is_event' that indicates whether each date is a significant event
data['is_major_event_day'] = data['Date'].apply(lambda x: 1 if x in events else 0)
data['is_commercial_event_day'] = data['Date'].apply(lambda x: 1 if x in commercial_events else 0)

data

,Date,Sessions,Bounce rate,Seven-day active users,28-day active users,New users,Scrolled users,Engaged sessions,Views,Event count,...,Sessions other,Sessions referral,season,commercial_season,week_of_month,day_of_week_nr,is_weekend,is_holiday,is_major_event_day,is_commercial_event_day
0,2024-01-04,982,0.362525,780,780,444,NaN,626,5111,19589,...,0.0,50.0,0.000000,0.000000,1,3,0,0,0,0
1,2024-01-05,887,0.363021,1433,1433,427,NaN,565,5000,17829,...,0.0,45.0,0.000000,0.000000,1,4,0,0,0,0
2,2024-01-06,426,0.415493,1748,1748,256,NaN,249,1920,6913,...,0.0,6.0,0.000000,-1.047198,1,5,1,0,0,0
3,2024-01-07,446,0.482063,2081,2081,261,NaN,231,2129,8848,...,0.0,11.0,0.000000,-1.047198,2,6,1,0,0,0
4,2024-01-08,981,0.387360,2749,2749,428,NaN,601,4745,16848,...,0.0,56.0,0.000000,-1.047198,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,2025-03-01,1298,0.554700,7458,29843,1091,325.0,578,3392,14491,...,0.0,39.0,1.570796,-1.047198,1,5,1,0,0,0
423,2025-03-02,1002,0.469062,7404,29753,692,337.0,532,3582,14427,...,0.0,32.0,1.570796,-1.047198,2,6,1,0,0,0
424,2025-03-03,1451,0.352860,7158,29497,674,710.0,939,6884,26862,...,0.0,181.0,1.570796,-1.047198,2,0,0,0,0,0
425,2025-03-04,1538,0.379714,7047,29494,791,723.0,954,6664,25808,...,0.0,147.0,1.570796,-1.047198,2,1,0,0,0,0


# drop irrelevant or redundant columns

In [105]:
data_no_nulls=data.dropna(axis=0, how='any')

## low variance variables

In [106]:
from sklearn.feature_selection import VarianceThreshold

variable_types: dict[str, list] = get_variable_types(data_no_nulls)

numeric: list[str] = [col for col in variable_types["numeric"] if col != "Date"]

# Remove the 'Date' column before applying VarianceThreshold
data_no_nulls = data_no_nulls[numeric]


# Remove features with very low variance
selector = VarianceThreshold(threshold=0.1)  # Ajuste o threshold conforme necessário
X_reduced = selector.fit_transform(data_no_nulls)

# Get selected features
selected_features = data_no_nulls.columns[selector.get_support()]
print(selected_features)

Index(['Sessions', 'Seven-day active users', '28-day active users',
       'New users', 'Scrolled users', 'Engaged sessions', 'Views',
       'Event count', 'Events per session', 'Item-list view events',
       'Item view events', 'Add to baskets', 'Checkouts',
       'E-commerce purchases', 'Average purchase revenue', 'Purchase revenue',
       'First time purchasers', 'Average session duration', 'ARPU', 'ARPPU',
       'Views per user', 'Total users', 'Promotion views', 'Promotion clicks',
       'One-day active users', 'Sessions Mobile',
       'Seven-day active users Mobile', '28-day active users Mobile',
       'New users Mobile', 'Scrolled users Mobile', 'Engaged sessions Mobile',
       'Views Mobile', 'Event count Mobile', 'Events per session Mobile',
       'Item-list view events Mobile', 'Item view events Mobile',
       'Add to baskets Mobile', 'Checkouts Mobile',
       'E-commerce purchases Mobile', 'Average purchase revenue Mobile',
       'Purchase revenue Mobile', 'Firs

In [107]:
# Print initial features
print("Initial features:\n", data_no_nulls.columns.tolist())

# Compute variance for each feature
feature_variance = data_no_nulls.var().round(1)
print("\nFeature Variance:\n", feature_variance)

# Define threshold (adjust based on the dataset)
threshold = 0.1
selector = VarianceThreshold(threshold=threshold)

# Fit and transform the dataset
X_reduced = selector.fit_transform(data_no_nulls)

# Get selected features
initial_features = list(set(data_no_nulls.columns))
removed_features = list(set(data_no_nulls.columns) - set(selected_features))
selected_features = data_no_nulls.columns[selector.get_support()]


# Print results
print("Initial Features df_prep:\n", initial_features)
print("\nRemoved Features (low variance < {:.3f}):\n".format(threshold), removed_features)
print("\nFinal Features list:\n", selected_features.tolist())


Initial features:
 ['Sessions', 'Bounce rate', 'Seven-day active users', '28-day active users', 'New users', 'Scrolled users', 'Engaged sessions', 'Views', 'Event count', 'Events per session', 'Item-list view events', 'Item view events', 'Item promotion click-through rate', 'Add to baskets', 'Checkouts', 'E-commerce purchases', 'Average purchase revenue', 'Purchase revenue', 'First time purchasers', 'Average session duration', 'ARPU', 'ARPPU', 'DAU/MAU', 'DAU/WAU', 'Sessions per user', 'Views per user', 'Total users', 'Promotion views', 'Promotion clicks', 'One-day active users', 'Sessions Mobile', 'Bounce rate Mobile', 'Seven-day active users Mobile', '28-day active users Mobile', 'New users Mobile', 'Scrolled users Mobile', 'Engaged sessions Mobile', 'Views Mobile', 'Event count Mobile', 'Events per session Mobile', 'Item-list view events Mobile', 'Item view events Mobile', 'Item promotion click-through rate Mobile', 'Add to baskets Mobile', 'Checkouts Mobile', 'E-commerce purchases 

In [108]:
data = data.drop(columns=removed_features)

data

,Date,Sessions,Seven-day active users,28-day active users,New users,Scrolled users,Engaged sessions,Views,Event count,Events per session,...,Sessions other,Sessions referral,season,commercial_season,week_of_month,day_of_week_nr,is_weekend,is_holiday,is_major_event_day,is_commercial_event_day
0,2024-01-04,982,780,780,444,NaN,626,5111,19589,19.948065,...,0.0,50.0,0.000000,0.000000,1,3,0,0,0,0
1,2024-01-05,887,1433,1433,427,NaN,565,5000,17829,20.100338,...,0.0,45.0,0.000000,0.000000,1,4,0,0,0,0
2,2024-01-06,426,1748,1748,256,NaN,249,1920,6913,16.227700,...,0.0,6.0,0.000000,-1.047198,1,5,1,0,0,0
3,2024-01-07,446,2081,2081,261,NaN,231,2129,8848,19.838565,...,0.0,11.0,0.000000,-1.047198,2,6,1,0,0,0
4,2024-01-08,981,2749,2749,428,NaN,601,4745,16848,17.174312,...,0.0,56.0,0.000000,-1.047198,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,2025-03-01,1298,7458,29843,1091,325.0,578,3392,14491,11.164099,...,0.0,39.0,1.570796,-1.047198,1,5,1,0,0,0
423,2025-03-02,1002,7404,29753,692,337.0,532,3582,14427,14.398204,...,0.0,32.0,1.570796,-1.047198,2,6,1,0,0,0
424,2025-03-03,1451,7158,29497,674,710.0,939,6884,26862,18.512750,...,0.0,181.0,1.570796,-1.047198,2,0,0,0,0,0
425,2025-03-04,1538,7047,29494,791,723.0,954,6664,25808,16.780234,...,0.0,147.0,1.570796,-1.047198,2,1,0,0,0,0


## redundant variables

In [109]:
data_no_nulls=data.dropna(axis=0, how='any')

In [110]:
import numpy as np

variable_types: dict[str, list] = get_variable_types(data_no_nulls)

numeric: list[str] = [col for col in variable_types["numeric"] if col != "Date"]

data_no_nulls = data_no_nulls[numeric]

# Calculate correlation matrix
corr_matrix = data_no_nulls[numeric].corr()

# Identify features highly correlated with the target variable "Sessions" (correlation > 0.9)
high_corr_features = corr_matrix['Sessions'][corr_matrix['Sessions'] > 0.9].index.tolist()

# Remove the target variable itself from the list
high_corr_features.remove('Sessions')

# Print the highly correlated features
print(f"Features too highly correlated with 'Sessions' to drop: {high_corr_features}")



Features too highly correlated with 'Sessions' to drop: ['New users', 'Scrolled users', 'Engaged sessions', 'Event count', 'One-day active users']


In [111]:
# Drop the correlated features
data = data.drop(columns=high_corr_features)

# Print remaining features count
print(f"Remaining features: {len(data.columns)}")
print(f"Remaining features: {data.columns.tolist()}")

Remaining features: 56
Remaining features: ['Date', 'Sessions', 'Seven-day active users', '28-day active users', 'Views', 'Events per session', 'Item-list view events', 'Item view events', 'Add to baskets', 'Checkouts', 'E-commerce purchases', 'Average purchase revenue', 'Purchase revenue', 'First time purchasers', 'Average session duration', 'ARPU', 'ARPPU', 'Views per user', 'Total users', 'Promotion views', 'Promotion clicks', 'Sessions Mobile', 'Seven-day active users Mobile', '28-day active users Mobile', 'New users Mobile', 'Scrolled users Mobile', 'Engaged sessions Mobile', 'Views Mobile', 'Event count Mobile', 'Events per session Mobile', 'Item-list view events Mobile', 'Item view events Mobile', 'Add to baskets Mobile', 'Checkouts Mobile', 'E-commerce purchases Mobile', 'Average purchase revenue Mobile', 'Purchase revenue Mobile', 'First time purchasers Mobile', 'Average session duration Mobile', 'Sessions (none)', 'Sessions (not set)', 'Sessions affiliate', 'Sessions cpc', 'S

# save file

In [112]:
if test_data==False:
    data.to_csv('data/df_merch_0_2024_agg_time_pre_proc.csv',index=False)